# Diet Problem Large Scale
We wish to minimize the cost of meeting our daily requirements of a set of nutrients $\mathcal{N}$, with a diet restricted to a set of foods $\mathcal{F}$. 
Each food $f\in\mathcal{F}$ has a given cost per weight unit, $C_f$, and at most $U_f$ units are available. Furthermore, each food $f\in\mathcal{F}$ provides a given quantity $A_{fn}$ of nutrient $n\in\mathcal{N}$ and our daily diet requires at least a quantity $R_n$ of nutrient $n$. Finally, there are two special subset of foods $\mathcal{F}^A$ and $\mathcal{F}^B$: if any of the foods in $\mathcal{F}^A$ is consumed, then one can consume at most $B$, in total, of the foods in $\mathcal{F}^B$. 

- Provide a mathematical formulation of the problem
- Find a solution to the problem using Gurobi


## Instance

In [1]:
import random as r
r.seed(1)
n_foods = 50
n_nutrients = 10
content = {(f,n): 100 + 20 * r.random() for f in range(n_foods) for n in range(n_nutrients)}
requirement = [10 + 200 * r.random() for n in range(n_nutrients)]
costs = [50 + 50 * r.random() for f in range(n_foods)]
max_amount = [20 + 40 * r.random() for f in range(n_foods)]
foods_a = [f for f in range(n_foods) if r.random() > 0.9]
foods_b = [f for f in range(n_foods) if (r.random() > 0.8) and f not in foods_a]
max_b = r.randint(50,70)

# Solution

Let $x_f$ be the amount of food $f$ consumed, $y_f$ an indicator variable which takes value $1$ if food $f$ is consumed, $0$ otherwise, $z^A$ an indicator variable taking value $1$ if any food in $\mathcal{F}^A$ is consumed.
$$
\begin{align*}
  \min ~& \sum_{f\in\mathcal{F}}C_fx_f \\
  s.t. ~& \sum_{f\in\mathcal{F}}A_{fn}x_f\geq R_n & \forall n\in \mathcal{N} \\
   & x_f\leq U_fy_f& \forall f\in\mathcal{F} \\
   & y_{f} \leq z^A & \forall f \in \mathcal{F}^A\\
   & \sum_{f\in\mathcal{F}^B}x_f + Mz^a\leq B+M\\
   & x_f\geq 0 & \forall f\in \mathcal{F}\\
   &y_f \in\{0,1\}& \forall f\in \mathcal{F}\\
   &z^A \in \{0,1\}
\end{align*}
$$
Here $M$ is a constant whose value can be $M=\sum_{f\in\mathcal{F}^B}U_f-B$.

## Problem class
It is good practice to divide the code into individual specialized units which in turn allow high reusability and scalability. To do this we will use the Object Oriented paradigm where objects represent individual units and classes represent templates for objects of the same type. 
In this spirit we will create a *class* which models the optimization problem we are solving: not the mathematical problem, but the real life problem, with its constituting elements (e.g, the diet problem, made of foods, nutrients, costs, etc.). An *object* of a class represents a specific instance of the problem (e.g., a diet problem with meat and vegetables as foods, vitamins and proteins as nutrients and a specific value for the parameters). 

Let us start by creating a class for the diet problem we described above.

In [2]:
class DietProblem:
    
    def __init__(self,n_foods:int,n_nutrients:int,
                 costs:list,max_amount:list,
                 requirements:list, content:dict,
                 foods_a:list,foods_b:list,max_b:int):
        self.n_foods = n_foods
        self.n_nutrients = n_nutrients
        self.costs = costs
        self.max_amount = max_amount
        self.requirements = requirements
        self.content = content
        self.foods_a = foods_a
        self.foods_b = foods_b
        self.max_b = max_b

In this simple version, the class does nothing else than storing the data of the problem. This is particularly advantageous if we need to pass the data to several places (as we will see when we implement algorithms): we can pass an object of this class instead of all the different lists and dictionaries. In addition, in more complicated circumstances, the constructor of a class (the `__init__()` method) may do additional operations such as generating additional parameters from the ones passed as input. 

Let us now create an instance of the `DietProblem` class.

In [4]:
p = DietProblem(n_foods,n_nutrients,costs,max_amount,requirement,content,foods_a,foods_b,max_b)
p.n_foods, p.max_b, p.costs

(50,
 56,
 [74.61495664458549,
  82.38341209210915,
  68.87791059255065,
  60.19570252183399,
  50.19378289387779,
  63.881062580471095,
  79.90820993568305,
  94.08314665353481,
  91.4710624994265,
  75.54801039355965,
  99.35090725247136,
  73.07904869349017,
  91.72967430834191,
  70.44826706404857,
  87.23153088693658,
  99.37958456113408,
  65.26682961839882,
  58.51564126066421,
  81.00168543638304,
  76.54780901870173,
  67.97110159925771,
  50.17596210485256,
  69.45813208049022,
  71.29347360518301,
  70.26260358691596,
  93.06226544887753,
  79.2214013541066,
  86.69153962265838,
  94.89545858185552,
  87.43867317875689,
  74.63510259525235,
  87.28841701434231,
  82.0177700247632,
  82.43727173316702,
  81.48376793443275,
  70.34994874942464,
  81.4631015643794,
  81.68662554728138,
  96.85589797694888,
  89.12368426854115,
  92.31340333005454,
  88.37498950712862,
  90.76629309955143,
  80.27311973651054,
  67.47250441933419,
  63.22916291590682,
  85.40100135324147,
  93.6

In [6]:
new_costs = [40 + 40 * r.random() for f in range(n_foods)]
p2 = DietProblem(n_foods,n_nutrients,new_costs,max_amount,requirement,content,foods_a,foods_b,max_b)
p2.n_foods, p2.max_b, p2.costs

(50,
 56,
 [72.58725507516019,
  60.949855145225094,
  62.34584740381881,
  73.76023287617747,
  63.25454434790798,
  60.803147416001266,
  41.21073481345724,
  79.19139082181962,
  79.645518449344,
  69.72714309603927,
  47.998967877135236,
  55.569541311585205,
  52.96881529610756,
  56.38011614840605,
  45.054670147569674,
  42.59451480018336,
  52.0198454901423,
  71.95857007834823,
  61.34727129746978,
  56.703529148793514,
  52.75197218436483,
  50.90677569070121,
  69.94946114674622,
  60.80410435827281,
  40.344454206862565,
  44.87455938719406,
  52.684431050544376,
  69.07222906517175,
  71.38826707826985,
  62.9243996944565,
  58.07255058867084,
  51.18449106207426,
  58.16568580653808,
  54.566616394405735,
  69.66549127392534,
  55.221109037288805,
  75.58327732248402,
  43.1268146402506,
  63.15869772474994,
  42.2440240188185,
  41.9490455161666,
  59.686900846840146,
  74.12397503629253,
  50.07674121378319,
  49.81459698167926,
  62.9513552029875,
  53.5459546332293,
 

Now all the data is stored in the object `p`. If we needed to work with two different diet problems, with different data, we could simply create two instances of the `DietProblem` class, say `p1` and `p2`.

## Model class
In a similar way we create a class which represents the template of a model for the diet problem. 
Its constructor will take as input an instance of the `DietProblem` class and generate its mathematical model. 
Let us start by importing some useful stuff.

In [7]:
from gurobipy import GRB, Model, quicksum

In [8]:
class DietProblemModel:
    
    def __init__(self,p:DietProblem):
        self.m = Model('diet_problem')
        self.p = p

        # Create the variables
        self.x = self.m.addVars(self.p.n_foods,lb=0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="x")
        self.y = self.m.addVars(self.p.n_foods,vtype=GRB.BINARY,name="y")
        self.z = self.m.addVar(vtype=GRB.BINARY,name="z")
        
        # Create the objective function
        self.m.setObjective(self.x.prod(self.p.costs),sense= GRB.MINIMIZE)

        # Create the constraints. 
        self.m.addConstrs(
            quicksum([self.p.content[f,n] * self.x[n] for f in range(self.p.n_foods)])
            >= self.p.requirements[n] for n in range(self.p.n_nutrients))
        self.m.addConstrs(self.x[f] <= self.p.max_amount[f] * self.y[f] for f in range(self.p.n_foods))
        self.m.addConstrs(self.y[f] <= self.z for f in self.p.foods_a)
        # We calculate M
        M = sum([self.p.max_amount[f] for f in self.p.foods_b]) - self.p.max_b
        self.m.addConstr(quicksum([self.x[f] for f in self.p.foods_b]) + M * self.z <= self.p.max_b + M)

    def solve(self):
        self.m.optimize()
    
    def print_solution(self):
        print('Objective value: %g' % self.m.objVal)
        for f in range(self.p.n_foods):
            print('food %g: quantity %g, used %g' % (f, self.x[f].x, self.y[f].x))
        print('Used foods from set A? %g' %  (self.z.x))


We can now create a model for a specific instance of the diet problem

In [109]:
m = DietProblemModel(p)

In [110]:
m.solve()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 128 physical cores, 128 logical processors, using up to 32 threads
Academic license - for non-commercial use only - registered to gp@math.ku.dk
Optimize a model with 65 rows, 101 columns and 130 nonzeros
Model fingerprint: 0x05904a9d
Variable types: 50 continuous, 51 integer (51 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+03]
  Objective range  [5e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 4e+02]
Presolve removed 65 rows and 101 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 128 available processors)

Solution count 1: 13.7015 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.370146303512e+01, best bound 1.370146303512e+01, gap 0.0000%


In [111]:
m.print_solution()

Objective value: 13.7015
food 0: quantity 0.0200819, used 1
food 1: quantity 0.0148526, used 1
food 2: quantity 0.0211277, used 1
food 3: quantity 0.00184519, used 1
food 4: quantity 0.0178834, used 1
food 5: quantity 0.0180968, used 1
food 6: quantity 0.0128106, used 1
food 7: quantity 0.0165215, used 1
food 8: quantity 0.0303572, used 1
food 9: quantity 0.0265339, used 1
food 10: quantity 0, used 1
food 11: quantity 0, used 1
food 12: quantity 0, used 1
food 13: quantity 0, used 1
food 14: quantity 0, used 1
food 15: quantity 0, used 1
food 16: quantity 0, used 1
food 17: quantity 0, used 1
food 18: quantity 0, used 1
food 19: quantity 0, used 1
food 20: quantity 0, used 1
food 21: quantity 0, used 1
food 22: quantity 0, used 1
food 23: quantity 0, used 1
food 24: quantity 0, used 1
food 25: quantity 0, used 1
food 26: quantity 0, used 1
food 27: quantity 0, used 1
food 28: quantity 0, used 1
food 29: quantity 0, used 1
food 30: quantity 0, used 0
food 31: quantity 0, used 1
food 32:

In [9]:
m2 = DietProblemModel(p2)

Restricted license - for non-production use only - expires 2023-10-25


In [10]:
m2.solve()

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 128 physical cores, 128 logical processors, using up to 32 threads
Optimize a model with 65 rows, 101 columns and 130 nonzeros
Model fingerprint: 0xdef2bbc3
Variable types: 50 continuous, 51 integer (51 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+03]
  Objective range  [4e+01, 8e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 4e+02]
Presolve removed 65 rows and 101 columns
Presolve time: 0.05s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.10 seconds (0.00 work units)
Thread count was 1 (of 128 available processors)

Solution count 1: 12.1521 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.215207829636e+01, best bound 1.215207829636e+01, gap 0.0000%


In [11]:
m2.print_solution()

Objective value: 12.1521
food 0: quantity 0.0200819, used 1
food 1: quantity 0.0148526, used 1
food 2: quantity 0.0211277, used 1
food 3: quantity 0.00184519, used 1
food 4: quantity 0.0178834, used 1
food 5: quantity 0.0180968, used 1
food 6: quantity 0.0128106, used 1
food 7: quantity 0.0165215, used 1
food 8: quantity 0.0303572, used 1
food 9: quantity 0.0265339, used 1
food 10: quantity 0, used 1
food 11: quantity 0, used 1
food 12: quantity 0, used 1
food 13: quantity 0, used 1
food 14: quantity 0, used 1
food 15: quantity 0, used 1
food 16: quantity 0, used 1
food 17: quantity 0, used 1
food 18: quantity 0, used 1
food 19: quantity 0, used 1
food 20: quantity 0, used 1
food 21: quantity 0, used 1
food 22: quantity 0, used 1
food 23: quantity 0, used 1
food 24: quantity 0, used 1
food 25: quantity 0, used 1
food 26: quantity 0, used 1
food 27: quantity 0, used 1
food 28: quantity 0, used 1
food 29: quantity 0, used 1
food 30: quantity 0, used 0
food 31: quantity 0, used 1
food 32: